In [1]:
import tensorflow as tf
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [1]:
import tensorflow.keras as keras

In [2]:
from keras.applications.mobilenet import MobileNet

In [3]:
# Loading the Dataset and getting size of it
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

print("Size of the train images:", x_train.shape)
print("Size of the test images:", x_test.shape)

Size of the train images: (50000, 32, 32, 3)
Size of the test images: (10000, 32, 32, 3)


In [4]:
def preprocess_data(X, Y):
        """ This method has the preprocess to train a model """
        X = X.astype('float32')
        X_p = keras.applications.mobilenet.preprocess_input(X)
        Y_p = keras.utils.to_categorical(Y, 10)
        return(X_p, Y_p)


In [5]:
x_train, y_train = preprocess_data(x_train, y_train)
x_test, y_test = preprocess_data(x_test, y_test)

if __name__ == "__main__":
    
    base_model = keras.applications.mobilenet.MobileNet(weights='imagenet',include_top=False)
    
    image_size = (224, 224)
    
    model= keras.Sequential()
    keras.layers.Resizing(image_size[0], image_size[1], interpolation="bilinear", input_shape=x_train.shape[1:]),
    model.add(base_model) 
    model.add(keras.layers.GlobalAveragePooling2D())
    model.add(keras.layers.Dense(1024,activation=('relu')))
    model.add(keras.layers.Dense(512,activation=('relu')))
    model.add(keras.layers.Dense(256,activation=('relu')))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(128,activation=('relu')))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(10,activation=('softmax')))
    

    callback = []
    
    def decay(epoch):
        """ This method create the alpha"""
        return 0.001 / (1 + 1 * 30)
    
    callback += [keras.callbacks.LearningRateScheduler(decay, verbose=1)]
    callback += [keras.callbacks.ModelCheckpoint('mobilenet_cifar10_v2.h5',
                                             save_best_only=True,
                                             mode='min'
                                             )]
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(x=x_train, y=y_train,
               batch_size=128,
               validation_data=(x_test, y_test),
               epochs=10, shuffle=True,
               callbacks=callback,
               verbose=1
               )

In [9]:
if __name__ == "__main__":
    
    base_model = keras.applications.mobilenet.MobileNet(weights='imagenet',include_top=False)
    
    image_size = (224, 224)
    
    model = keras.Sequential()
    model.add(keras.layers.Resizing(image_size[0], image_size[1], interpolation="bilinear", input_shape=x_train.shape[1:]))
    model.add(base_model)
    model.add(keras.layers.GlobalAveragePooling2D())
    #model.add(keras.layers.Dense(1024, activation='relu'))
    model.add(keras.layers.Dense(512, activation='relu'))
    model.add(keras.layers.Dense(256, activation='relu'))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(128, activation='relu'))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(10, activation='softmax'))
    
    callback = []
    
    def decay(epoch):
        """ This method create the alpha"""
        return 0.001 / (1 + 1 * 30)
    
    callback += [keras.callbacks.LearningRateScheduler(decay, verbose=1)]
    callback += [keras.callbacks.ModelCheckpoint('mobilenet_cifar10.h5',
                                             save_best_only=True,
                                             mode='min'
                                             )]
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(x=x_train, y=y_train,
               batch_size=32,
               validation_data=(x_test, y_test),
               epochs=10, shuffle=True,
               callbacks=callback,
               verbose=1
               )


Epoch 1: LearningRateScheduler setting learning rate to 3.2258064516129034e-05.
Epoch 1/10


2024-03-20 22:02:28.435719: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1563/1563 [==============================] - ETA: 0s - loss: 2.6189 - accuracy: 0.4809

2024-03-20 22:07:49.951173: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1563/1563 [==============================] - 338s 215ms/step - loss: 2.6189 - accuracy: 0.4809 - val_loss: 0.9778 - val_accuracy: 0.7616 - lr: 3.2258e-05

Epoch 2: LearningRateScheduler setting learning rate to 3.2258064516129034e-05.
Epoch 2/10
1563/1563 [==============================] - 330s 211ms/step - loss: 1.0271 - accuracy: 0.7785 - val_loss: 0.7296 - val_accuracy: 0.8310 - lr: 3.2258e-05

Epoch 3: LearningRateScheduler setting learning rate to 3.2258064516129034e-05.
Epoch 3/10
1563/1563 [==============================] - 332s 212ms/step - loss: 0.6905 - accuracy: 0.8487 - val_loss: 0.8903 - val_accuracy: 0.8150 - lr: 3.2258e-05

Epoch 4: LearningRateScheduler setting learning rate to 3.2258064516129034e-05.
Epoch 4/10
1563/1563 [==============================] - 335s 214ms/step - loss: 0.5399 - accuracy: 0.8825 - val_loss: 0.7558 - val_accuracy: 0.8485 - lr: 3.2258e-05

Epoch 5: LearningRateScheduler setting learning rate to 3.2258064516129034e-05.
Epoch 5/10
1563/1563 [=====

In [10]:
from keras.models import load_model

model = load_model('mobilenet_cifar10.h5')

In [11]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

predictions = model.predict(x_test)
predicted_labels = np.argmax(predictions, axis=1)

# Calculating metrics
print("Confusion Matrix:")
conf_matrix = confusion_matrix(np.argmax(y_test, axis=1), predicted_labels)
print(conf_matrix)

print("\nClassification Report:")
print(classification_report(np.argmax(y_test, axis=1), predicted_labels))


2024-03-21 00:03:38.689907: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


313/313 [==============================] - 15s 47ms/step
Confusion Matrix:
[[931   0   6   3   2   6   5   4  28  15]
 [  3 933   0   0   0   1   1   0  16  46]
 [ 26   0 810  22  27  70  29   6  10   0]
 [  3   0   3 612   5 343  23   2   7   2]
 [  2   0   4   1 913  57  10  12   1   0]
 [  0   0   2   6   5 976   4   6   1   0]
 [  3   0   2   3   1  20 968   0   3   0]
 [  8   0   3   2  17  76   3 890   0   1]
 [ 10   3   2   2   1   5   1   0 967   9]
 [  3  12   0   1   0   3   0   0   8 973]]

Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.93      0.94      1000
           1       0.98      0.93      0.96      1000
           2       0.97      0.81      0.88      1000
           3       0.94      0.61      0.74      1000
           4       0.94      0.91      0.93      1000
           5       0.63      0.98      0.76      1000
           6       0.93      0.97      0.95      1000
           7       0.97      0.89    